In [1]:
import pandas as pd
import numpy as np
AR_all_features = pd.read_csv('AR-all-desc.csv')

/ihome/gidakwo/anaconda3/envs/ml_env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1240) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Drop colmns used for arriving at consensus_act, not needed for modeling
cols_to_drop = np.r_[2:7, 8:16]
AR_all_features.drop(AR_all_features.columns[cols_to_drop],axis=1,inplace=True)
AR_all_features.set_index('PUBCHEM_SID', inplace=True)

In [3]:
AR_all_features.shape

(10251, 18049)

In [56]:
AR_all_features.head()

,CID,consensus_act,Vina_E,nALA,nARG,nASN,nASP,nCYS,nGLN,nGLU,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
PUBCHEM_SID,,,,,,,,,,,,,,,,,,,,,
144212021,4,inactive,-3.7,2,2,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
144209154,6,inconclusive,-6.5,2,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
144213119,6,inconclusive,-6.5,2,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
144209664,11,inconclusive,-2.8,2,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
144208509,13,inactive,-5.5,1,2,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#Mode function to assign an activity to a CID based on the most occuring activity for that CID.
#GOAL: A compound with conflicting activity, e.g. a compound listed four times (twice as inactive and twice as antagonist) 
#      should be deleted
#      For other cases, the most occuring activity (consensus_act) is retained leaving only one entry for every compound
def calc_mode(x):
    m = pd.Series.mode(x)
    if len(m) == 1: 
        return m

In [23]:
AR_no_dup = AR_all_features.groupby('CID').consensus_act.apply(calc_mode).reset_index(level=1, drop=True).reset_index()

In [24]:
AR_no_dup.shape

(7665, 2)

In [31]:
AR_no_dup.head()

,CID,consensus_act
0,4,inactive
1,6,inconclusive
2,11,inconclusive
3,13,inactive
4,33,inactive


In [57]:
#AR_no_dup.to_csv('AR_no_dup.csv')
#Now, Get the features from the main data set
AR_unique = pd.merge(AR_no_dup, AR_all_features,  how='left', left_on=['CID', 'consensus_act'], 
                 right_on = ['CID', 'consensus_act']).drop_duplicates(subset=['CID'])

In [58]:
AR_unique.to_csv('AR_unique.csv')

In [55]:
AR_unique.head()

,CID,consensus_act,Vina_E,nALA,nARG,nASN,nASP,nCYS,nGLN,nGLU,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
0,4,inactive,-3.7,2,2,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
1,6,inconclusive,-6.5,2,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,11,inconclusive,-2.8,2,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,13,inactive,-5.5,1,2,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
6,33,inactive,-3.0,2,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [59]:
docking_data = AR_unique.iloc[:,:82]
docking_data.shape

(7665, 82)

In [60]:
cols_ = np.r_[0,1, 82:18049]
conventional_data = AR_unique.iloc[:,cols_]
conventional_data.shape

(7665, 17969)

In [61]:
conventional_data.head()

,CID,consensus_act,nAcid.1,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
0,4,inactive,0,-1.1977,1.434485,20.7955,13.183137,0,0,14,...,0,0,0,0,0,0,0,0,0,0
1,6,inconclusive,0,0.6511,0.423931,11.3872,20.148379,6,6,16,...,0,0,0,0,0,0,0,0,0,0
3,11,inconclusive,0,1.5118,2.285539,20.6556,10.547172,0,0,8,...,0,0,0,0,0,0,0,0,0,0
4,13,inactive,0,2.2329,4.985842,16.8468,19.100379,6,6,12,...,0,0,0,0,0,0,0,0,0,0
6,33,inactive,0,0.3501,0.122570,16.4715,8.502379,0,0,7,...,0,0,0,0,0,0,0,0,0,0


In [62]:
docking_data.head()

,CID,consensus_act,Vina_E,nALA,nARG,nASN,nASP,nCYS,nGLN,nGLU,...,ang_764.876.Lig,ang_764.891.Lig,ang_876.741.Lig,ang_876.752.Lig,ang_876.764.Lig,ang_876.891.Lig,ang_891.741.Lig,ang_891.752.Lig,ang_891.764.Lig,ang_891.876.Lig
0,4,inactive,-3.7,2,2,1,0,0,1,2,...,111.731584,109.797326,75.339212,131.569629,111.731584,54.989050,82.412365,158.882117,109.797326,54.989050
1,6,inconclusive,-6.5,2,1,1,0,0,1,0,...,136.651847,114.995905,88.013564,144.164280,136.651847,63.079820,87.218246,149.868930,114.995905,63.079820
3,11,inconclusive,-2.8,2,1,0,0,0,1,1,...,116.721034,113.873075,78.759612,132.499168,116.721034,57.321547,85.691687,158.622200,113.873075,57.321547
4,13,inactive,-5.5,1,2,1,0,0,1,1,...,120.607886,111.580964,86.402695,135.307220,120.607886,60.508969,90.409877,151.622228,111.580964,60.508969
6,33,inactive,-3.0,2,1,0,0,0,1,1,...,110.113264,117.707140,75.116626,125.248307,110.113264,56.504527,86.794681,162.219940,117.707140,56.504527


In [63]:
docking_data.to_csv('docking_data_unique.csv')
conventional_data.to_csv('conventional_data_unique.csv')